# FLANER model evaluation

### Config

In [8]:
CHECKPOINT_PATH = '/home/ubuntu/gil/flanner_trainer/models/v3/batch_size_64/checkpoint-897'
TEST_LABELED_DATA_PATH = '/home/ubuntu/gil/flanner_trainer/val_prompt_dataset_final_corrected15.csv'
TEST_SET_PREDICTIONS_PATH = '/home/ubuntu/gil/flanner_trainer/'
TASK_PREFIX = 'ner_last_name:'
OUTPUT_DIR = '/home/ubuntu/gil/flanner_trainer/models/ctranslate_model'

In [2]:
DATA_CLASSES_TAGS = {
    "LN": "last_name"
}

GROUND_TRUTH_DATA_CLASSES_TAGS = {
    "last_name": "last_name"  

}

In [3]:
ENTITY_DELIMITER = " #### "
VALUES_DELIMITER = ","

In [4]:
from model_config import *

### Dependencies

In [14]:
import pandas as pd
import re
from ds_commons.flanner_utils import convert_model_to_ct2, load_ct2_model, ct2_inference
from transformers import AutoTokenizer

In [9]:
convert_model_to_ct2(model_dir=CHECKPOINT_PATH, output_dir=OUTPUT_DIR)

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.80s/it]


In [12]:
ct_model = load_ct2_model(model_dir_path=OUTPUT_DIR, device='cuda')

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

/home/ubuntu/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
CONTROL_CHARS_PATTERN = re.compile("[\u0000-\u0008\u000b\u000c\u000e-\u001f\u007f-\u00a0]+")

def clean_text(context):
    context = str(context)
    context = context.replace('\n', ' ')
    context = context.replace('\t', ' ')
    context = context.replace('\\r\\n', ' ')
    context = CONTROL_CHARS_PATTERN.sub(' ', context).strip()
    context = re.sub(' +', ' ', context)
    return context


df = pd.read_csv(TEST_LABELED_DATA_PATH)
df['clean_context'] = df['context'].apply(lambda x: clean_text(x))
model_inputs = df['clean_context'].to_list()
model_inputs = [TASK_PREFIX+x for x in model_inputs]

In [19]:
model_inputs[0:5]

['ner_last_name:Client: Hall-Johnston Walker, Rojas and Wells',
 'ner_last_name:C12004000197OP010412Brooklyn NY2131WVitaliy Gavrylyuk WCLVitaliy Gavrylyuk 11218X',
 'ner_last_name:Sent: Tuesday, April 01, 2014 10:32 AM To: OGrady.Elizabeth; Agencyauthos',
 'ner_last_name:Statement Period Feb 1 - Feb 28, 2021 Relationship Manager Cabrera,Ana',
 'ner_last_name:this proponent again? Yes, currently working with Ellis Don on construction a $370+M DBF']

In [22]:
res = ct2_inference(
    model_input=model_inputs,
    tokenizer=tokenizer,
    translator=ct_model,
    max_length=1000, #  what should it be?
    generation_max_length=GENERATION_MAX_LEN,
    batch_size=BATCH_SIZE
)

In [24]:
res[0:10]

["LN:['Walker', 'Rojas', 'Wells']",
 'LN:[]',
 'LN:[]',
 "LN:['Cabrera']",
 'LN:[]',
 'LN:[]',
 "LN:['BOGDAN', 'RADU']",
 'LN:[]',
 'LN:[]',
 "LN:['Law', 'Crosby']"]